In [89]:
#********************************************* Secondary Allocatin NOTEBOOK **********************************************
# This notebook will read in 
# 1. Source Sheet
# 2. Allocation rule sheet
# 3. CC Master Sheet
# Find this data in three files
#*****************************************************************************************************************
import pandas as pd
%pylab inline


#set the Environment
env_name = 'RAND'


if env_name == 'RAND':
    
    #set the Base
    base_path = '/home/sm/work/RCL/Groceries/'
    #This is the Randfontein Groceries environment Set any variables here
    env = base_path+'Budgets/'
    
    #Routes - Components and their routs
    qty_allocation_file = {'file_data':{'file':env+'MASTER/Gro Activity quantity v6.xlsx',
                                             'sheet_name':0,
                                             'skip_rows':1,'skip_footer':0},
                               'col_map':{},
                               'out_cols':['Cost Centre','Activities', 'Activity Type', 'e313 - ActQty']
 
                               }
    
    allocation_rules_file = {'file_data':{'file':env+'MASTER/Allocation Rules.xlsx',
                                             'sheet_name':0,
                                             'skip_rows':1,'skip_footer':0},
                               'col_map':{},
                               'out_cols':['Src_CC', 'Src_CC_Desc', 'Group', 'GroupWA', 
                                           'Rec_CC', 'Rec_CC_Desc','CC_WA', 'Variator']
                               }
    
    allocation_groups_file = {'file_data':{'file':env+'MASTER/Allocation Rules.xlsx',
                                             'sheet_name':1,
                                             'skip_rows':0,'skip_footer':0},
                               'col_map':{},
                               
                               }
    
    cc_master_file = {'file_data':{'file':env+'MASTER/CC attributes upload 29 09 2016.xlsx',
                                             'sheet_name':0,
                                             'skip_rows':0,'skip_footer':0},
                               'col_map':{}
                               }
    
else:
    raise ValueError('Please set an environment')
    
path_out = env+'Output/' 
path_master = env+'MASTER/'


############################################################## FUNCTION
##
#  Support Functions
#
# The function is the core function used to load a file from config
# 
##
#############
def write_to_excel(path,data_frame,sheet_name='Sheet1', startrow = 0):
    #pd_test[['NCA Number','Date raised','NCA Details','Machine Name','Authorised By','Nature of Pollutant']]
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    path = str(path).replace('.xlsx','',0)
    writer = pd.ExcelWriter(path+'.xlsx', engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object.
    data_frame.to_excel(writer, sheet_name=sheet_name, startrow = startrow)

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

############################################################## FUNCTION
##
##Load a master file
#
# The function is the core function used to load a file from config
# 
##
#############
def load_master_file(**kwargs):
    file_master = kwargs.get('file_master_data',{})
    
    if len(file_master)==0:
        raise ValueError('Missing Input Parameters')
    #Load and map the source CC allocation file
    file_data = file_master.get('file_data',{})
    
    if len(file_data)==0:
        raise ValueError('Missing File Data to load file')
    
    
    
    data = pd.read_excel(file_data.get('file',''),header=0,
                         skiprows=file_data.get('skip_rows',0),
                         skip_footer=file_data.get('skip_footer',0),
                        sheetname= file_data.get('sheet_name',0))
    data.rename(columns=file_master.get('col_map',{}),inplace=True)
    outcols = file_master.get('out_cols','')
    if len(outcols)>0:
        data = data[outcols]
    
    return data

############################################################## FUNCTION
##
##Function to create the macs file
##
#############
def get_allocation_file(**kwargs):
    #qty_allocation = 
    
    allo_groups = kwargs.get('allo_groups','')
    allo_rules = kwargs.get('allo_rules','')
    allo_qty = kwargs.get('allo_qty','')
    cc_master = kwargs.get('cc_master','')
    
    Year = kwargs.get('Year','')
    ValSys = kwargs.get('ValSys','')
    version = kwargs.get('version','')
    
    if len(Year)==0 or len(ValSys)==0 or len(version)==0:
        raise ValueError('Missing Static Data')
    
    if len(allocation_groups)==0 or len(allocation_rules)==0 or len(allo_qty)==0 :
        raise ValueError('Missing Allocation Data!')
    
    if len(cc_master)==0:
        raise ValueError('Missing Cost Center Data!')
    cc_master['Cost Element'] = cc_master['Cost Element'].fillna('-')
    cc_master['Sec CEL'] = cc_master[['Plant of assembly e314','Cost Element']].apply(lambda r:'{}-{}'.format(r[1],r[0]),axis=1)
    cc_cols = ['CC', 'Description', 'Activities','Sec CEL']
    cc_master = cc_master[cc_cols]

    # Use the allocation Qty and rules to preduce the secondary allocation file for macs
    
    
    #Business Rules for the Rules file.
    #1. The allocation qty will be split by the group first. The sum of GroupWA on the group should be 1
    allo_wa_result = allo_groups.groupby('Group ID')['GR_WA'].sum().reset_index()
    if pd.DataFrame.any(allo_wa_result['GR_WA']!=1.0):
        raise ValueError('Some Groups Are not allocating Correctly',allo_wa_result[allo_wa_result['GR_WA']!=1.0])
        
    #2. The sum of the const center weigted avarage CC-WA should be 1
    allo_wa_result = allo_rules.groupby(['Src_CC','Group','GroupWA'])['CC_WA'].sum().reset_index()
    if pd.DataFrame.any((allo_wa_result['CC_WA']>1.0000001)|(allo_wa_result['CC_WA']<0.99999999)):
        raise ValueError('Some Alloctions not sum to 1',allo_wa_result[(allo_wa_result['CC_WA']>1.0000001)|(allo_wa_result['CC_WA']<0.99999999)])
    
    merged_rules = allo_rules.merge (allo_groups,
                             left_on=['Group','GroupWA'],
                             right_on=['Group ID','Group Name'],how='left')
    if pd.DataFrame.any(pd.isnull(merged_rules['GR_WA'])):
        raise ValueError('Missing Groups',merged_rules[pd.isnull(merged_rules['GR_WA'])][['Group','GroupWA']])
        
    #calc the value of to allocate on each line as x*Group_WA*CC_Wa
    merged_rules['eff_WA'] = merged_rules[['CC_WA','GR_WA']].apply(lambda r: r[0]*r[1],axis=1)
    
    merged_rules =  merged_rules.merge(allo_qty,
                              left_on='Src_CC', 
                              right_on='Cost Centre')
    merged_rules['eff_qty'] = merged_rules[['eff_WA','e313 - ActQty']].apply(lambda r: r[0]*r[1],axis=1)
    merged_rules['Variable'] = merged_rules[['eff_qty','Variator']].apply(lambda r: r[0]*r[1]/100,axis=1)
    merged_rules['Fixed'] = merged_rules[['eff_qty','Variator']].apply(lambda r: r[0]*(1-r[1]/100),axis=1)
    
    #Generate the Macs Output file and return it
    in_cols = ['Src_CC', 'Rec_CC','Variator','eff_qty','Variable', 'Fixed']
    merged_rules = merged_rules[in_cols].merge(cc_master,
                                       left_on='Src_CC',
                                       right_on='CC')
    
    merged_rules.rename(columns = {'Description':'Sending CostCentre Description',
                                   'Src_CC':'Sending Cost Centre Number',
                                   'Activities':'Sending ActType',
                                   'eff_qty':'Total allocation quantity'
                                  },inplace=True)
    
    merged_rules = merged_rules.merge(cc_master[['CC','Description','Activities']],
                                       left_on='Rec_CC',
                                       right_on='CC')
    merged_rules.rename(columns = {'Description':'Receiving CostCentre Name',
                                   'Rec_CC':'Receiving CostCentre',
                                   'Activities':'Receiving ActType'
                                  },inplace=True)
    
    merged_rules['Year'] = Year
    merged_rules['ValSys'] = ValSys
    merged_rules['version'] = version
    
    outcols = ['Year','ValSys','version',
               'Sending Cost Centre Number','Sending CostCentre Description',
               'Sending ActType','Sec CEL','Receiving CostCentre','Receiving CostCentre Name',
               'Receiving ActType','Total allocation quantity','Variable','Fixed']

    out_df =  merged_rules[outcols]
    out_df = out_df[out_df['Total allocation quantity'] > 0].groupby(['Year','ValSys', 'version', 'Sending Cost Centre Number',
       'Sending CostCentre Description', 'Sending ActType', 'Sec CEL',
       'Receiving CostCentre', 'Receiving CostCentre Name',
       'Receiving ActType'])[['Total allocation quantity', 'Variable', 'Fixed']].sum().reset_index().set_index(outcols[0])
    return out_df
print('Done')

Populating the interactive namespace from numpy and matplotlib
Done


In [90]:
############################################################## 
##
## Load The required Files
##
##############################################################
allocation_rules = load_master_file(file_master_data=allocation_rules_file)
allocation_groups = load_master_file(file_master_data=allocation_groups_file)
allocation_qty = load_master_file(file_master_data=qty_allocation_file)
cc_master = load_master_file(file_master_data=cc_master_file)

In [91]:
############################################################## 
##
## Run The script from here
##
##############################################################
Year = '2017'
ValSys = '1002'
version = '3'

#Change this to the next file name. The file will go into /Output folder
output_file_name = 'Allocation Qty v7'

# Change this to False if you don't want to write an output file
write_output = True

#This is the function that calc the file data and place it in allocations variable
allocations = get_allocation_file(allo_groups = allocation_groups,
                    allo_rules = allocation_rules,
                    allo_qty = allocation_qty,
                    cc_master = cc_master,
                    Year = Year,
                    ValSys = ValSys,
                    version = version
                   )   

#Optionally write output to a file
if write_output:
    write_to_excel(path_out+output_file_name, 
                   allocations,
                   startrow=1
                  )
    print('Wrote New Output')

#Print the top part of the Out put
allocations.head()

Wrote New Output


,ValSys,version,Sending Cost Centre Number,Sending CostCentre Description,Sending ActType,Sec CEL,Receiving CostCentre,Receiving CostCentre Name,Receiving ActType,Total allocation quantity,Variable,Fixed
Year,,,,,,,,,,,,
2017,1002,3,63563,RY: Electricity Distribution,KWH,990030-NL,63544,RY: Plant General Overheads,AICost,141326.181830,141326.181830,0.0
2017,1002,3,63563,RY: Electricity Distribution,KWH,990030-NL,63602,RY: Roaster,MachHrs,598604.768316,598604.768316,0.0
2017,1002,3,63563,RY: Electricity Distribution,KWH,990030-NL,63604,RY: Crunchy Nib Grinder,MachHrs,3503.117597,3503.117597,0.0
2017,1002,3,63563,RY: Electricity Distribution,KWH,990030-NL,63605,RY: Grinder B1 and B2,MachHrs,130299.169144,130299.169144,0.0
2017,1002,3,63563,RY: Electricity Distribution,KWH,990030-NL,63606,RY: Urschell Grinder (Yum Yum),MachHrs,239261.148097,239261.148097,0.0
